<a href="https://colab.research.google.com/github/jglaser/gigadocking_notebooks/blob/main/extract_top.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
project_id = 'cmb-covid19-discovery-collab'

In [30]:
%%time
%%bigquery --project $project_id df
SELECT scores.name, score, smiles.smiles FROM mpro_5r84.confs_all AS confs
INNER JOIN mpro_5r84.scores_rank0 AS scores ON scores.name = confs.name
INNER JOIN vf_2018.smiles ON smiles.name = confs.name
WHERE score IS NOT NULL and rank = 19 # 0
ORDER BY score 
LIMIT 10000

CPU times: user 1.38 s, sys: 127 ms, total: 1.51 s
Wall time: 1min 19s


In [34]:
df

,name,score,smiles
0,Z71169259_1_T1,-12.06,O=C(NC[C@@H]1CC2c3ccccc3C1c1ccccc21)c1ccc2c(c1...
1,Z2150409399_1_T1,-11.62,O=C(Nc1cccc(c1)C#Cc1cccs1)N1CCC2(C1)Oc1ccccc1O2
2,Z2910540677_24_T1,-11.58,CC1CN(CCN(C1)C(=O)[C@H]1[C@H]2CCC(=O)[C@H]12)C...
3,Z1283902295_2_T1,-11.52,CC1(C)[C@@H](NC(=O)N2Cc3ccccc3-c3ccccc3C2)[C@@...
4,Z2079388592_1_T1,-11.43,O=C(N1CCC2(CCOCC2)CC1)c1cc(nc2ccccc12)-c1ccc2O...
...,...,...,...
9995,Z1560332643_2_T1,-9.87,Brc1ccccc1[C@H]1N(CCNC1=O)C(=O)Nc1ccc2CCC(=O)N...
9996,Z2179071547_1_T3,-9.87,OC(=C(C([S-])=NCC1=CC=CC=C1)[N+]1=CC(O)=CC(=C1...
9997,Z1383808022_1_T1,-9.87,Cc1cc(nn1-c1ccccc1)C(=O)Nc1ccc2C(=O)c3ccccc3C(...
9998,PV-001879650024_2_T1,-9.87,C[C@@H](Oc1ccc2ccccc2c1)C(=O)NNC(=O)c1ccc(o1)-...


In [32]:
df.to_parquet('smiles_adgpu_top10k.parquet')

In [33]:
from google.colab import files
files.download('smiles_adgpu_top10k.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# also extract conformation
%%time
%%bigquery --project $project_id df
SELECT scores.name, conf, score  FROM mpro_5r84.confs_all AS confs
INNER JOIN mpro_5r84.scores_rank0 AS scores ON scores.name = confs.name
WHERE rank = 19 # 0
ORDER BY score 
LIMIT 10000

Executing query with job ID: 3e0b61f3-652b-4e27-bc79-4a4bbc4a8bad
Query executing: 1655.63s

KeyboardInterrupt: ignored